# Quick Inference Model using a single Perceptron

Lets delete any saved model if it exists

In [ ]:
!rm mymodel.hd5
!mkdir -p temp/images

In [ ]:
# Uncomment if needed
#!pip install -U tensorflow matplotlib numpy pandas

In [ ]:
%matplotlib inline

Load the libraries and if a model exists load that, if not we'll load a saved model. It is a single perceptron using a linear activation (default) and a Stochastic Gradient Descent optimizer.

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
#NN def image
#import pydot
from IPython.display import Image
import time

#Careful with the below
import warnings
warnings.filterwarnings("ignore")

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

try:
  model=tf.keras.models.load_model('./mymodel.hd5')
  print("model loaded")
except:
  print("Creating new model")
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(1, input_dim=1))
  model.compile(loss='mean_squared_error',optimizer='sgd')
 

In [ ]:
def predict(model=None,xs=None,ys=None,predict=None,line=True):
    """
    Given a model and data build a prediction and graph it's accuracy
    """
    to_predict = predict
    predicted = model.predict(to_predict)[:,0]
    x_show=np.concatenate((xs, to_predict), axis=0)
    y_show=np.concatenate((ys, predicted),axis=0)
    c_show=np.concatenate((np.repeat('blue',len(xs+1)),np.repeat('red',len(to_predict+1))))
    data=pd.DataFrame({'x':x_show,'y':y_show,'colors':c_show})

    data.plot.scatter(x='x',y='y',c=c_show)
    #z = np.polyfit(data['x'], data['y'], 1)
    if line:
      z1 = np.polyfit(to_predict, predicted, 1)
      p1 = np.poly1d(z1)
      z2 = np.polyfit(xs, ys, 1)
      p2 = np.poly1d(z2)
      plt.plot(x_show,p2(x_show),"b-")
      plt.plot(data['x'],p1(data['x']),"r--")

      # the line equation:
      print("y=%.6fx+(%.6f)"%(z1[0],z1[1]))
    plt.show();

In [ ]:
def draw_model(model,image_name=None):
    """
    Given a NN model, draw the rough diagram of the neural network
    """
    G = nx.DiGraph()
    model_config=model.get_config()
    layer=[]
    #print('Model has {} layers'.format(len(model_config['layers'])))
    layer.append('layer 0_0')
    for i in range(len(model_config['layers'])):
        try: 
            inputs=model_config['layers'][i-1]['config']['units']
        except:
            inputs=1
        new_layer=list(range(inputs))
        for j in range(inputs):
            try:
                for k in range(model_config['layers'][i]['config']['units']):
                    new_layer.append('layer {}_{}'.format(i+1,k))
                    G.add_edge('layer {}_{}'.format(i,j),'layer {}_{}'.format(i+1,k))
                    #print('layer {}_{} to layer {}_{}'.format(i,j,i+1,k))
            except:
                continue
        layer.append(new_layer)
    A = nx.nx_agraph.to_agraph(G)
    for i in range(len(layer)):
        A.add_subgraph(layer[i],rank='same')                       
    A.draw(image_name,prog='dot')
    

Here is the data we are going to load. Its a straight linear relationship.

In [ ]:
size=7
xs = np.linspace(-1,size-1,num=size)
ys = np.add(2*xs-1,0.3*(2*np.random.normal(size=size)-1))

In [ ]:
plt.plot(xs,ys,'o')

In [ ]:
model.fit(xs,ys,epochs=1,verbose=0)

In [ ]:
model.get_config()

## Prediction
Lets test the prediction

In [ ]:
to_predict = np.array([10,11,12,13])
predicted = model.predict(to_predict)[:,0]
print(predicted)

In [ ]:
tf.keras.utils.plot_model(
    model,
    to_file='temp/images/model.png',
    show_shapes=True,
    show_layer_names=True,
    rankdir='TB'
)

In [ ]:
predict(model=model,xs=xs,ys=ys,predict=np.array([10,11,12,13]))

Not great, but not bad for 5 tries. Lets do another 10 passes

In [ ]:
model.fit(xs,ys,epochs=10,verbose=0)

predict(model=model,xs=xs,ys=ys,predict=np.array([10,11,12,13]))

In [ ]:
model.fit(xs,ys,epochs=10,verbose=0)

predict(model=model,xs=xs,ys=ys,predict=np.array([10,11,12,13]))

In [ ]:
model.fit(xs,ys,epochs=10,verbose=0)

predict(model=model,xs=xs,ys=ys,predict=np.array([10,11,12,13]))

In [ ]:
model.fit(xs,ys,epochs=10,verbose=0)

predict(model=model,xs=xs,ys=ys,predict=np.array([10,11,12,13]))

## full tilt, 10000 epochs

In [ ]:
model.fit(xs,ys,epochs=10000,verbose=0)

predict(model=model,xs=xs,ys=ys,predict=np.array([10,11,12,13]))

## Save the model

Now we can save the model for use later. Yay!!

In [ ]:
#model.save('mymodel.hd5')
#print("./model saved")

# More complicated Data

Lets do a test with parabolic data

In [ ]:
size=7
xs = np.linspace(-1,size-1,num=size)
ys = np.add(2*xs*xs-10*xs-1,0.3*(2*np.random.normal(size=size)-1))
plt.plot(xs,ys,'o')

In [ ]:
model.fit(xs,ys,epochs=100,verbose=0)
predict(model=model,xs=xs,ys=ys,predict=np.array([4.1,4.2,4.3,4.4]))

Lines don't work, so let's make the model detect two parts. so instead of 1 neuron in the layer let's have 2

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(2, input_dim=1))
model.add(tf.keras.layers.Dense(1))
model.compile(loss='mean_squared_error',optimizer='sgd')

In [ ]:
time_start = time.time()
model.fit(xs,ys,epochs=100,verbose=0)
time_end = time.time()
print(time_end-time_start)
predict(model=model,xs=xs,ys=ys,predict=np.array(np.linspace(-1,size-1,num=100)),line=False)

In [ ]:
tf.keras.utils.plot_model(
    model,
    to_file='temp/images/model.png',
    show_shapes=True,
    show_layer_names=True,
    rankdir='TB'
)

A Line plus a line is a line, we need something non-linear. Lets add an activation function. In this scenario we'll use the ReLU activation, anything +ve is linear, anything -ve is zero.

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(2, input_dim=1,activation=tf.keras.activations.relu))
model.add(tf.keras.layers.Dense(1))
model.compile(loss='mean_squared_error',optimizer='sgd')

In [ ]:
tf.keras.utils.plot_model(
    model,
    to_file='temp/images/model.png',
    show_shapes=True,
    show_layer_names=True,
    rankdir='TB'
)

In [ ]:
epochs=1000
time_start = time.time()
model.fit(xs,ys,epochs=epochs,verbose=0)
time_end = time.time()
print((time_end-time_start)/epochs)
predict(model=model,xs=xs,ys=ys,predict=np.array(np.linspace(-1,size-1,num=100)),line=False)

Better, but lets see if we can pick up more complex shapes by making it 5 input layers (5 line segments)

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(5, input_dim=1,activation=tf.keras.activations.relu))
model.add(tf.keras.layers.Dense(1))
model.compile(loss='mean_squared_error',optimizer='sgd')

epochs=1000
time_start = time.time()
model.fit(xs,ys,epochs=epochs,verbose=0)
time_end = time.time()
print((time_end-time_start)/epochs)
predict(model=model,xs=xs,ys=ys,predict=np.array(np.linspace(-1,size-1,num=100)),line=False)

Will 10 neurons make it better?

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(10, input_dim=1,activation=tf.keras.activations.relu))
model.add(tf.keras.layers.Dense(1))
model.compile(loss='mean_squared_error',optimizer='sgd')

In [ ]:
epochs=1000
time_start = time.time()
model.fit(xs,ys,epochs=epochs,verbose=0)
time_end = time.time()
print((time_end-time_start)/epochs)
predict(model=model,xs=xs,ys=ys,predict=np.array(np.linspace(-1,size-1,num=100)),line=False)

Lets get better data

In [ ]:
datapoints=50
xs = np.linspace(-1,size-1,num=datapoints)
ys = np.add(2*xs*xs-10*xs-1,0.3*(2*np.random.normal(size=datapoints)-1))
plt.plot(xs,ys,'o')

In [ ]:
epochs=1000
time_start = time.time()
model.fit(xs,ys,epochs=epochs,verbose=0)
time_end = time.time()
print((time_end-time_start)/epochs)
predict(model=model,xs=xs,ys=ys,predict=np.array(np.linspace(-1,size-1,num=100)),line=False)

Lets add a new layer

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(5, input_dim=1,activation=tf.keras.activations.relu))
model.add(tf.keras.layers.Dense(5, input_dim=1,activation=tf.keras.activations.relu))
model.add(tf.keras.layers.Dense(1))
model.compile(loss='mean_squared_error',optimizer='sgd')

In [ ]:
epochs=10000
time_start = time.time()
model.fit(xs,ys,epochs=epochs,verbose=0)
time_end = time.time()
print((time_end-time_start)/epochs)
predict(model=model,xs=xs,ys=ys,predict=np.array(np.linspace(-1,size-1,num=100)),line=False)

In [ ]:
print(model.summary())

Lets Change to Sigmoid and see if we can get a smoother curve

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(5, input_dim=1,activation=tf.keras.activations.sigmoid))
model.add(tf.keras.layers.Dense(5, input_dim=1,activation=tf.keras.activations.sigmoid))
model.add(tf.keras.layers.Dense(1))
model.compile(loss='mean_squared_error',optimizer='sgd')

#draw_model(model,'temp/images/image_5.png')
#Image(filename='temp/images/image_5.png') 

In [ ]:
print(model.summary())

In [ ]:
epochs=10000
time_start = time.time()
model.fit(xs,ys,epochs=epochs,verbose=0)
time_end = time.time()
print((time_end-time_start)/epochs)
predict(model=model,xs=xs,ys=ys,predict=np.array(np.linspace(-1,size-1,num=100)),line=False)

Lets change the initialization functions (random normal limited (he_normal)

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(5, input_dim=1,activation=tf.keras.activations.sigmoid,kernel_initializer='he_normal',
                bias_initializer='zeros'))
model.add(tf.keras.layers.Dense(5, input_dim=1,activation=tf.keras.activations.sigmoid,kernel_initializer='he_normal',
                bias_initializer='zeros'))
model.add(tf.keras.layers.Dense(1))
model.compile(loss='mean_squared_error',optimizer='sgd')

In [ ]:
epochs=10000
time_start = time.time()
model.fit(xs,ys,epochs=epochs,verbose=0)
time_end = time.time()
print((time_end-time_start)/epochs)
predict(model=model,xs=xs,ys=ys,predict=np.array(np.linspace(-1,size-1,num=100)),line=False)

# 2D Data?

In [ ]:
from mpl_toolkits import mplot3d

import numpy as np
import matplotlib.pyplot as plt
import random

%matplotlib inline

datapoints=100
xs = np.linspace(-1,1,num=datapoints)
ys = np.linspace(-1,1,num=datapoints)

def f(x):
    return (x[0]**2)+(x[1]**3)+0.05*np.reshape((2*np.random.normal(size=datapoints*datapoints)-1),(datapoints,datapoints))

fig = plt.figure()
X, Y = np.meshgrid(xs, ys)
Z = f([X, Y])

#ax = plt.axes(projection='3d')
#ax.plot_surface(X, Y, Z, cmap='viridis')
#ax.view_init(30,45)
#plt.draw()
plt.contourf(X,Y,Z)

In [ ]:
def plot_df(X,Y,Z,title=None,xlim=None,ylim=None,zlim=None):
    fig = plt.figure()

    ax = plt.axes(projection='3d')
    ax.plot_surface(X, Y, Z, cmap='viridis')
    ax.view_init(30,30)
    #plt.draw()
    if xlim:
        plt.xlim(xlim[0], xlim[1])
    if ylim:
        plt.ylim(ylim[0], ylim[1])
    if zlim:
        ax.set_zlim(zlim[0], zlim[1])
    plt.scatter(X,Y,Z)
    plt.title(title)

In [ ]:
plot_df(X,Y,(Z),title='Target',xlim=[-1,1],ylim=[-1,1],zlim=[-1,2])

In [ ]:
np.shape(Z)

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(5,input_dim=2,activation=tf.keras.activations.sigmoid,kernel_initializer='he_normal',
                bias_initializer='zeros'))
model.add(tf.keras.layers.Dense(5,activation=tf.keras.activations.relu,kernel_initializer='he_normal',
                bias_initializer='zeros'))
model.add(tf.keras.layers.Dense(1))
model.compile(loss='mean_squared_error',optimizer='sgd')

In [ ]:
print(model.summary())

In [ ]:
df=pd.DataFrame({
    'x':list(X.reshape(-1)),
    'y':list(Y.reshape(-1)),
#    'z':f([X.reshape(-1),Y.reshape(-1)])})
    'z':Z.reshape(-1)})

In [ ]:
df.describe()

In [ ]:
features=df[['x','y']].values
labels=df[['z']].values

In [ ]:
features.shape

In [ ]:
epochs=100
time_start = time.time()
history = model.fit(features,labels,batch_size=32,epochs=epochs,verbose=0)
time_end = time.time()
print(time_end-time_start)

In [ ]:
tf.keras.utils.plot_model(
    model,
    to_file='temp/images/model.png',
    show_shapes=True,
    show_layer_names=True,
    rankdir='TB'
)

In [ ]:
#model.predict([1,1])[:,0].reshape(1,-1)
Zp=model.predict(features)[:,0].reshape(100,100)
Zp.shape

In [ ]:
model.predict(features)[:,0]

In [ ]:
plot_df(X,Y,(Z),title='Target',xlim=[-1,1],ylim=[-1,1],zlim=[-1,2])
plot_df(X,Y,(Zp),title='Predicted',xlim=[-1,1],ylim=[-1,1],zlim=[-1,2])
plot_df(X,Y,(Z-Zp),title='Error',xlim=[-1,1],ylim=[-1,1],zlim=[-1,2])

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(25,input_dim=2,activation=tf.keras.activations.sigmoid,kernel_initializer='he_normal',
                bias_initializer='zeros'))
model.add(tf.keras.layers.Dense(25,activation=tf.keras.activations.sigmoid,kernel_initializer='he_normal',
                bias_initializer='zeros'))
model.add(tf.keras.layers.Dense(1))
model.compile(loss='mean_squared_error',optimizer='sgd')

In [ ]:
epochs=1000
time_start = time.time()
history = model.fit(features,labels,batch_size=32,epochs=epochs,verbose=0)
time_end = time.time()
print(time_end-time_start)

In [ ]:
tf.keras.utils.plot_model(
    model,
    to_file='temp/images/model.png',
    show_shapes=True,
    show_layer_names=True,
    rankdir='TB'
)

In [ ]:
time_start = time.time()
Zp=model.predict(features)[:,0].reshape(100,100)
time_end = time.time()
print(time_end-time_start)

In [ ]:
plot_df(X,Y,(Z),title='Target',xlim=[-1,1],ylim=[-1,1],zlim=[-1,2])
plot_df(X,Y,(Zp),title='Predicted',xlim=[-1,1],ylim=[-1,1],zlim=[-1,2])
plot_df(X,Y,(Z-Zp),title='Error',xlim=[-1,1],ylim=[-1,1],zlim=[-1,2])